In [1]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import json
import os
import random
import glob
# from detectron_pro import detectron_mask_img,detectron_mask_img_composite
import shutil
import cv2

import random
from tqdm import tqdm

In [2]:

# to get location that stickimg will sticked on jpg_dir center or random
def center_location(jpg_dir,stickimg_dir):
    im = np.array(Image.open(jpg_dir), dtype=np.uint8)
    x_center,y_center = im.shape[1]/2,im.shape[0]/2
    im_stick = np.array(Image.open(stickimg_dir), dtype=np.uint8)
    im_stick_shape = im_stick.shape
    bd_box_x,bd_box_y = x_center-(im_stick_shape[1]/2),y_center-(im_stick_shape[0]/2)
    bd_box_length,bd_box_height =im_stick_shape[1],im_stick_shape[0]
    return bd_box_x,bd_box_y,bd_box_length,bd_box_height

def random_location(jpg_dir,stickimg_dir):
    im = np.array(Image.open(jpg_dir), dtype=np.uint8)
    # x boundary
    #rangeX = 400
    #x_left_bound,x_right_bound = rangeX, im.shape[1]-rangeX
    x_left_bound,x_right_bound = 128,512
    x_center = random.randrange(x_left_bound, x_right_bound)
    seq = [320,280,240,220,200]
    y_center = random.choice(seq)
    i_shape1 = [240,120]
    i_shape2 = [160,80]
    i_shape3 = [128,64]
    i_shape4 = [108,54]
    i_shape5 = [96,48]
    dict = {352:i_shape1,296:i_shape2 ,240:i_shape3,220:i_shape4,200:i_shape5}
    image_shape = dict[y_center]
    bd_box_x,bd_box_y = x_center-(image_shape[1]/2),y_center-(image_shape[0]/2)
    bd_box_length,bd_box_height =image_shape[1],image_shape[0]
    return bd_box_x,bd_box_y,bd_box_length,bd_box_height


#h_list = [200,220,240,296,352] # from top to buttom (center of people image)
#h_scale = [(48,96),(54,108),(64,128),(80,160),(120,240)] # top might be smaller

In [3]:
#it can generate json file defined by your street_jpg and people_jpg 

def create_json_file(jpg_dir,street_json,stickimg_dir,results_dir,function='center'):
    if function == 'center':
        bd_box_x,bd_box_y,bd_box_length,bd_box_height = center_location(jpg_dir,stickimg_dir)
    elif function == 'random':
        bd_box_x,bd_box_y,bd_box_length,bd_box_height = random_location(jpg_dir,stickimg_dir)
        
    input_file = open (street_json)
    json_array = json.load(input_file)
    
    data = []
    data.append({
    'end':0,
    'hide':0,
    'id':0,
    'init':0,
    'lbl':"pasted_person",
    'lock':0,
    'occl':0,
    'pos':[
    bd_box_x,
    bd_box_y,
    bd_box_length,
    bd_box_height],    
    'posv':[
    0,
    0,
    0,
    0],
    'str':0
                })
    
    if json_array != []:
        for item in json_array:
            data.append(item)
            
    with open(results_dir, 'w') as outfile:
        json.dump(data, outfile)

# Start create

In [4]:
origin_data_dir = '/root/notebooks/final/caltech_conver_data'

In [5]:
img_data = glob.glob(origin_data_dir+'/**/*.jpg', recursive=True)

# json_data = glob.glob(origin_data_dir+'/**/*.json', recursive=True)

In [6]:
img_data[:10]

['/root/notebooks/final/caltech_conver_data/set01/V000/images/I00103.jpg',
 '/root/notebooks/final/caltech_conver_data/set01/V000/images/I00199.jpg',
 '/root/notebooks/final/caltech_conver_data/set01/V000/images/I00229.jpg',
 '/root/notebooks/final/caltech_conver_data/set01/V000/images/I00255.jpg',
 '/root/notebooks/final/caltech_conver_data/set01/V000/images/I00268.jpg',
 '/root/notebooks/final/caltech_conver_data/set01/V000/images/I00512.jpg',
 '/root/notebooks/final/caltech_conver_data/set01/V000/images/I00562.jpg',
 '/root/notebooks/final/caltech_conver_data/set01/V000/images/I00780.jpg',
 '/root/notebooks/final/caltech_conver_data/set01/V000/images/I00782.jpg',
 '/root/notebooks/final/caltech_conver_data/set01/V000/images/I00947.jpg']

In [7]:
# json_data[:10]

In [8]:
# Image read dir
street_dir = '/root/notebooks/0858611-2/final_project/caltech_pedestrian_extractor/video_extractor/*'

# Image save dir
save_dir = '/root/notebooks/final/result_dataset_9'

# num_imgs = 10000
num_imgs = 'all'

In [9]:
# Check dir folder exit
# If not, create one
if os.path.exists(save_dir) == False:
    os.makedirs(save_dir)

for s in ['street', 'street_json']:
    if os.path.exists(os.path.join(save_dir, s)) == False:
        os.makedirs(os.path.join(save_dir, s))

In [10]:
#street_imgs = glob.glob(street_dir+'/**/*.jpg', recursive=True)
street_imgs = img_data

#street_imgs = random.shuffle(random.sample(street_imgs, 5000))
if num_imgs not in 'all':
    street_imgs = random.sample(street_imgs, num_imgs)

random.shuffle(street_imgs)

In [11]:
street_img_refined = []
# street_json_refined = []

In [12]:
len(street_imgs)

224191

In [13]:
pbar = tqdm(total=len(street_imgs))
for i in range(len(street_imgs)):
    
    #if (i%500==0):
        #print("Process (",i,"/",len(street_imgs),")  ","{:.2f}".format(100*i/len(street_imgs))," %")
    pbar.update()
        
    img_path = street_imgs[i]
    json_dir = img_path.replace('images', 'annotations')
    json_dir = json_dir.replace('jpg', 'json')
    
    input_file = open (json_dir)
    json_array = json.load(input_file)
    #if json_array != []:
    if json_array == []:
        street_img_refined.append(street_imgs[i])
    
    input_file.close()
pbar.close()

100%|██████████| 224191/224191 [10:18<00:00, 362.43it/s]


In [14]:
len(street_img_refined)

113278

In [15]:
pbar = tqdm(total=len(street_img_refined))
for i in range(len(street_img_refined)):
    pbar.update()
    
    img_path = street_img_refined[i]
    json_dir = img_path.replace('images', 'annotations')
    json_dir = json_dir.replace('jpg', 'json')
    
    shutil.copyfile(json_dir, save_dir+'/street_json/'+str('{0:06}'.format(i))+'.json')
    shutil.copyfile(img_path, save_dir+'/street/'+str('{0:06}'.format(i))+'.jpg')
    
pbar.close()

100%|██████████| 113278/113278 [43:12<00:00, 43.70it/s] 
